### Reto TC2008B.570 Modelación de Sistemas Multiagentes con Gráficas Computacionales  
####**Team**   
Silvana Dorita Ruiz Olivarría A01252346  
Alan Antonio de la Cruz Téllez A01280638  
Daniel Loredo Meléndez A01284184  
Mario Jáuregui Gómez A00832049  

####**Program's description**  
In this program there are two main agents, the Admin and the Cars. The Admin assigns each Car a spot in the plaza's parking lot taking into account the establishment that they want to visit to provide them with the closest available spot to it. Then, the car has to navigate through the parking lot's streets and then park on the assigned spot. After certain number of iterations or steps in the model according to the visited building, the car leaves the parking spot and the plaza.


In [ ]:
%pip install mesa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 2.4 MB/s eta 0:00:00


In [ ]:
from mesa import Agent, Model
from mesa.time import SimultaneousActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

from random import choice, randint
import numpy as np

In [ ]:
# Buildings and cars' colors
cars_colors = [ 'blue', 'green', 'orange', 'red', 'yellow']

# Coordinates of the parking spots
coord_parking = [
    [(1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11)],
    [(2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11)],
    [(4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11)],
    [(5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11)]
]

# Parking spots priority for the blue building
priority_blue = [
    [ 3,  2,  1,  4,  9, 19, 22, 26],
    [ 7,  5,  6,  8, 10, 20, 25, 29],
    [13, 11, 12, 14, 21, 24, 27, 31],
    [17, 15, 16, 18, 23, 28, 30, 32]
]

# Parking spots priority for the green building
priority_green = [
    [26, 22, 19,  9,  3,  2,  1,  4],
    [29, 25, 20, 10,  7,  5,  6,  8],
    [31, 27, 24, 21, 13, 11, 12, 14],
    [32, 30, 28, 23, 17, 15, 16, 18]
]

# Parking spots priority for the orange building
priority_orange = [
    [28, 24, 20, 16, 12,  8,  4,  1],
    [29, 25, 21, 17, 13,  9,  5,  2],
    [31, 27, 23, 19, 15, 11,  7,  3],
    [32, 30, 26, 22, 18, 14, 10,  6]
]

# Parking spots priority for the red building
priority_red = [
    [32, 30, 28, 23, 17, 15, 16, 18],
    [31, 27, 24, 21, 13, 11, 12, 14],
    [29, 25, 20, 10,  7,  5,  6,  8],
    [26, 22, 19,  9,  3,  1,  2,  4] 
]

# Parking spots priority for the yellow building
priority_yellow = [
    [17, 15, 16, 18, 23, 28, 30, 32],
    [13, 11, 12, 14, 21, 24, 27, 31],
    [ 7,  5,  6,  8, 10, 20, 25, 29],
    [ 3,  1,  2,  4,  9, 19, 22, 26]
]

# Coordinates of the plaza's streets
streets = [
    (3,0), (3,1), 
    (1,2), (2, 2), (3,2), (4,2), (5,2),
    (0,3), (0,4), (0,5), (0,6), (0,7), (0,8), (0,9), (0,10), (0,11), (0,12),
    (3,3), (3,4), (3,5), (3,6), (3,7), (3,8), (3,9), (3,10), (3,11), (3,12),
    (6,3), (6,4), (6,5), (6,6), (6,7), (6,8), (6,9), (6,10), (6,11), (6,12),
    (1,13), (2, 13), (3,13), (4,13), (5,13),
    (3,14), (3,15)
]

# Coordinates of the blocked cells
blocked_cells = [
    (0,0), (0,1), (0,2), (1,0), (1,1), (2,0),
    (4,0), (5,0), (6,0), (4,1), (5,1), (6,1), (6,2),
    (0,13), (0,14), (0,15), (1,14), (1,15), (2,14), (2,15),
    (4,14), (4,15), (5,14), (5,15), (6,13), (6,14), (6,15),
    (1,3), (2,3), (1,12), (2,12), (4,3), (5,3), (4, 12), (5,12)
]


def get_status_grid(model):
  """
  The function get_status_grid assigns each agent type 
  a number, which is used in the matplotlib visualization.
  It receives as a parameter the model and returns a matrix
  with the grid's dimensions and numbers 0 through 4 as values.
  """
  result = np.zeros((model.grid.width, model.grid.height))
  for (content, x, y) in model.grid.coord_iter():
    for obj in content:
      if isinstance(obj, Car):
        result[x][y] = 3 
      elif isinstance(obj, Street):
        result[x][y] = 1 
      elif isinstance(obj, ParkingSpot):
        result[x][y] = 0
      elif isinstance(obj, Admin):
        result[x][y] = 4
      elif isinstance(obj, BlockedCell):
        result[x][y] = 2
        
  return result


def verify_availability(closestSpot, model):
  """
  The function verify_availability identifies whether the
  closestSpot's state is 0 or not, meaning that it is free.
  It receives as parameters the coordinates of the closest 
  parking spot for a car and the model of the multiagent
  system. It return true if the spot is free and false if it 
  is not.
  """
  neighbors = model.grid.get_neighbors(
      (closestSpot),
      moore=False,
      include_center=True
  )
  for neighbor in neighbors:
    if isinstance(neighbor, ParkingSpot) and neighbor.pos == closestSpot:
      if neighbor.state == 0: # if the spot is free
        print('como el spot está libre, lo ocuparé')
        neighbor.state = 1 # now the spot has been taken
        return True
  return False


def find_min_priority(color_mat, target_priority, model):
  """ 
  The function find_min_priority determines if the targeted 
  priority parking spot is free. It receives as parameters
  the matrix of parking spots' priority according to the
  establishment that is being visited by the car, the target
  priority, and the model. It returns the parking coordinates 
  of the spot if it free and has the targeted priority, if 
  it has been taken, an empty tuple is returned.
  """
  for i in range(len(color_mat)):
    for j in range(len(color_mat[i])):
      if color_mat[i][j] == target_priority and verify_availability(coord_parking[i][j], model):
        print(f'find_min_priority es {target_priority}')
        return coord_parking[i][j]
  return ()
  


def closest_parking_spot(car, model):
  """
  The function closest_parking_spot finds the target priority 
  that the parking spot of the car being processed will have in the
  priority matrix corresponding to their destination, 
  checks the availability of said spot, if it is free its
  coordintes will be returned, if not, the target priority value
  will be increased by one and the process will be repeated.
  It receives as a parameter a car agent and the model of the 
  multiagent system. It returns the closests paring spot's coordiantes 
  or an empty tuple.
  """
  # 1. Check Cars Color
  # 2. Iterate over the Priority Parking Matrix of the specfic color
  # 3. Check for the availability on Parking based on Priority
  # 4. Return the first available spot for that specific color
  if car.color == 'blue':
    color_mat = priority_blue
  elif car.color == 'green':
    color_mat = priority_green
  elif car.color == 'orange':
    color_mat = priority_orange
  elif car.color == 'red':
    color_mat = priority_red
  else:
    color_mat = priority_yellow
  
  target_priority = 1
  while target_priority <= 32:
    print(f'target_priority es {target_priority}')
    closest_coord = find_min_priority(color_mat, target_priority, model)
    if len(closest_coord) == 2:
      print(f'{car.unique_id} tiene prioridad: {target_priority}')
      return closest_coord
    else:
      target_priority += 1
  return ()

# The ParkingSpot agents are located in the grid's cells where the agent cars can park
# They are located within a section of the parking lot (A, B, C or D).
# They can be either free o taken by a car. 
class ParkingSpot(Agent):
  def __init__(self, unique_id, model, section):
    super().__init__(unique_id, model)
    self.section = section # Section A, B, C or D
    self.state = 0 # 0: free, 1: taken


# The Street agent indicates which cells of the grid can have a car agent driving on them.
# Each Street agent can only hold a car each step.
class Street(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.state = 0 # 0: available, 1: unavailable


# The BlockedCell agent is just for visualization purposes.
# It shows the areas that cannot be occuppied by other agents such as cars, admin and streets.
class BlockedCell(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)


# The Car agent navigates through the plaza's streets and parking lot in order to 
# find a spot to park.
# It can be either waiting, moving or parked. It also is assigned a random color,
# which indicates it target establishment in the plaza.
class Car(Agent):
  # Constructor of Car class
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.state = 0 # 0: waiting, 1: moving, 2: parked
    self.next_state = self.state
    self.color = choice(cars_colors) # Color assignment     
    self.parking_coordinates = (0,0)
    self.next_pos = None
    self.direction = 0 # Car pointing => 0: Right, 1: Upwards, 2: Downwards, 3: Left
    self.next_direction = 0
    self.isActive = False
    self.next_isActive = False

    # Wait time (number of iterations the car should wait in the parking before exiting the Plaza)
    self.waiting_park_time = randint(20, 40)                 
    # Current Street
    self.streetOn = None

    self.to_free_flag_parking_spot = False
    self.parking_spot_to_free = None

    self.firstNeighbor = None
    self.adjacent_street_to_parking_spot_free = False

  # Step function of Car agent, determines the car movement on the streets
  def step(self):
    # Store current Spot Street to make it the spot available on the next movement 
    neighbors = self.model.grid.get_neighbors(
          (self.pos),
          moore = True,
          include_center = True,
      )
    for neighbor in neighbors:
      if isinstance(neighbor, Street) and self.pos == neighbor.pos: # Check if the next street is available
        self.streetOn = neighbor

    # -------------------------------------------
    # ---------MOVE CAR WITH ADMIN---------------
    # -------------------------------------------

    # If the car hasn't moved to the line
    if(self.state == 0 and self.parking_coordinates == (0,0) and self.pos == (3,0)):
      neighbors = self.model.grid.get_neighbors(
          (self.pos),
          moore = False,
          include_center = False,
      )
      # If there's no car in the line blocking the movement of this car, then the car can move to the entrance
      for neighbor in neighbors:
        if (isinstance(neighbor, Street)) and (neighbor.pos == (3,1)) and (neighbor.state == 0):
          # Only move one car to the line
          neighbor.state = 1
          self.next_state = 1
          self.next_pos = (3,1)
          self.next_isActive = True


    # -------------------------------------------
    # --------CHOOSING ROUTE OUT OF 3------------
    # -------------------------------------------

    # Move the car to the entrance of the Plaza parking
    elif(self.parking_coordinates != (0,0) and self.pos == (3,1)):
      self.state = 1

      # First parking line (Up)
      if(self.parking_coordinates[0] == 1):
        self.next_pos = (2,2)
        self.next_direction = 1
      # Second or third parking line (Center)
      elif(self.parking_coordinates[0] == 2 or self.parking_coordinates[0] == 4):
        self.next_pos = (3,3)
        self.next_direction = 0
      # Fourth parking line (Down)
      else:
        self.next_pos = (4,2)
        self.next_direction = 2

    # -------------------------------------------
    # ----------PARKING CAR (ACTION)-------------
    # -------------------------------------------


    # if the car is right next to the Parking Spot (one street), then it should park  
    elif(self.pos[1] == self.parking_coordinates[1] - 1 and self.parking_coordinates != (0,0) and self.state != 2 and self.waiting_park_time > 0):
        # Asign the direction of the parking
        if(self.parking_coordinates[0] == 1 or self.parking_coordinates[0] == 4):
          self.next_pos = (self.pos[0] + 1, self.pos[1] + 1)
          self.next_direction = 2
        else:
          self.next_pos = (self.pos[0] - 1, self.pos[1] + 1)
          self.next_direction = 1
        self.next_state = 2
  
    # -------------------------------------------
    # ---------EXITING PARKING SPOT--------------
    # -------------------------------------------

    # If the car has already passed N number of iterations parked, then it should exit the plaza
    elif(self.pos == self.parking_coordinates and self.waiting_park_time <= 0):
      self.to_free_flag_parking_spot = False
      self.adjacent_street_to_parking_spot_free = False
      for neighbor in neighbors:
          if isinstance(neighbor, Street):
            if self.parking_coordinates[0] == 1 or self.parking_coordinates[0] == 4:
              if(neighbor.pos[0] == self.pos[0] - 1 and neighbor.pos[1] == self.pos[1] and neighbor.state == 0):
                self.firstNeighbor = neighbor
                self.to_free_flag_parking_spot = True
                
            elif self.parking_coordinates[0] == 2 or self.parking_coordinates[0] == 5:
              if(neighbor.pos[0] == self.pos[0] + 1 and neighbor.pos[1] == self.pos[1] and neighbor.state == 0):
                self.firstNeighbor = neighbor
                self.to_free_flag_parking_spot = True


      for neighbor in neighbors:
        if self.to_free_flag_parking_spot and isinstance(neighbor, Street):
          if (self.parking_coordinates[0] == 1 or self.parking_coordinates[0] == 4):
            if (neighbor.pos[0] == self.pos[0] - 1) and (neighbor.pos[1] == self.pos[1] + 1) and (neighbor.state == 0):
              self.adjacent_street_to_parking_spot_free = True

              self.firstNeighbor.state = 1
              neighbor.state = 1
              self.next_pos = (self.pos[0] - 1, self.pos[1] + 1)
              self.next_direction = 0
              self.next_state = 1

          elif (self.parking_coordinates[0] == 2 or self.parking_coordinates[0] == 5):
            if (neighbor.pos[0] == self.pos[0] + 1) and (neighbor.pos[1] == self.pos[1] + 1) and (neighbor.state == 0):
              self.adjacent_street_to_parking_spot_free = True

              self.firstNeighbor.state = 1
              neighbor.state = 1
              self.next_pos = (self.pos[0] + 1, self.pos[1] + 1)
              self.next_direction = 0
              self.next_state = 1

    
    # -------------------------------------------
    # -------------CAR MOVEMENT------------------
    # -------------------------------------------
    
    # If the car has already chosen a direction and one of the 3 main streets
    else:

      if(self.state == 1 and self.pos[1] > 1):
        neighbors = self.model.grid.get_neighbors(
          (self.pos),
          moore = True,
          include_center = False,
        )

        # If the car is pointing upwards, check if the car can still move upwards
        if(self.direction == 1):
          street_neighbor = None

          for neighbor in neighbors:
            if isinstance(neighbor, Street):
              if (self.pos[0] == neighbor.pos[0] + 1) and (self.pos[1] == neighbor.pos[1]):
                street_neighbor = neighbor

                # Check if next position upwards is available
                if (street_neighbor.state == 0):
                  # Exiting Plaza via route 3
                  if (street_neighbor.pos == (3,13)):
                    neighbors_of_neighbor1 = self.model.grid.get_neighbors(
                      (neighbor.pos),
                      moore = False,
                      include_center = False,
                    )
                    for neighbor1 in neighbors_of_neighbor1:
                      if neighbor1.pos[0] == self.pos[0] - 1 and neighbor1.pos[1] == self.pos[1] + 1 and neighbor1.state == 0:
                        self.next_pos = neighbor1.pos
                        neighbor1.state = 1
                        self.next_direction = 0
                  else:
                    self.next_pos = street_neighbor.pos
                    street_neighbor.state = 1



          # No street upwards found, then move in diagonal and change direction to right
          if(self.next_pos == self.pos and street_neighbor == None):
            for neighbor in neighbors:
              if isinstance(neighbor, Street) and neighbor.pos == (self.pos[0] - 1, self.pos[1] + 1):
                # Check if next pos is available
                if (neighbor.state == 0):
                  self.next_pos = (self.pos[0] - 1, self.pos[1] + 1)
                  self.next_direction = 0
                  neighbor.state = 1
                  
                  
    
        # If the car is pointing downwards, check if the car can still move downwards
        elif(self.direction == 2):
          street_neighbor = None

          for neighbor in neighbors:
            if isinstance(neighbor, Street):
              if (self.pos[0] == neighbor.pos[0] - 1) and (self.pos[1] == neighbor.pos[1]):
                street_neighbor = neighbor

                # Check if next pos is available
                if(street_neighbor.state == 0):
                  # Exiting Plaza via route 1
                  if (street_neighbor.pos == (3,13)):
                    neighbors_of_neighbor2 = self.model.grid.get_neighbors(
                      (neighbor.pos),
                      moore = False,
                      include_center = False,
                    )
                    for neighbor2 in neighbors_of_neighbor2:
                      if neighbor2.pos[0] == self.pos[0] + 1 and neighbor2.pos[1] == self.pos[1] + 1 and neighbor2.state == 0:
                        self.next_pos = neighbor2.pos
                        neighbor2.state = 1
                        self.next_direction = 0
                  else:
                    self.next_pos = street_neighbor.pos
                    street_neighbor.state = 1

          # No street downwards found, then move in diagonal
          if(self.next_pos == self.pos and street_neighbor == None):
            for neighbor in neighbors:
              if isinstance(neighbor, Street) and neighbor.pos == (self.pos[0] + 1, self.pos[1] + 1):
                # Check if next pos is available
                if (neighbor.state == 0):
                  self.next_pos = (self.pos[0] + 1, self.pos[1] + 1)
                  self.next_direction = 0
                  neighbor.state = 1


        # If the car is pointing Right, check if it can keep moving right
        if(self.direction == 0):
          # If the car hasnt arrived to the parking spot, then continue moving
          for neighbor in neighbors:
            if isinstance(neighbor, Street):
              if self.pos[1] == neighbor.pos[1] - 1 and self.pos[0] == neighbor.pos[0]:
                # Check if next pos is available
                if (neighbor.state == 0):
                  
                  # Exiting Plaza via route 2
                  if (neighbor.pos == (3,13)):
                    neighbors_of_neighbor = self.model.grid.get_neighbors(
                      (neighbor.pos),
                      moore = False,
                      include_center = False,
                    )
                    for neighbor0 in neighbors_of_neighbor:
                      if neighbor0.pos[0] == self.pos[0] and neighbor0.pos[1] == self.pos[1] + 2 and neighbor0.state == 0:
                        self.next_pos = neighbor0.pos
                        neighbor0.state = 1
                  else:
                    self.next_pos = neighbor.pos
                    neighbor.state = 1

                    # If it's the last Cell, then maintain the neighbor state as empty to store all the cars that exits
                    if(neighbor.pos == (3,15)):
                      neighbor.state = 0
                      self.next_isActive = False
                else:
                  print(str(self.unique_id) + " -> Se detiene porque hay un auto en frente")

              # Condition for car that exits from the Plaza from the street 0 or 6
              elif(self.pos[1] == neighbor.pos[1] - 1 and self.pos[0] != neighbor.pos[0] and self.parking_coordinates[0] != 2  and self.parking_coordinates[0] != 4 ):
                if (neighbor.state == 0):
                  self.next_pos = neighbor.pos
                  if(self.pos[0] == neighbor.pos[0] + 1):
                    self.next_direction = 1
                  else:
                    self.next_direction = 2
                else:
                  print(str(self.unique_id) + " -> Se detiene porque hay un auto en frente")
      

      for neighbor in neighbors:
        if self.adjacent_street_to_parking_spot_free and isinstance(neighbor, ParkingSpot) and neighbor.pos == self.parking_coordinates:
          neighbor.state = 0

  
    # Logica para salir del estacionamiento
    if(self.state == 2):
      self.waiting_park_time = self.waiting_park_time - 1

          
  # Advance function of Car agent. It updates the values of the car attributes.
  def advance(self):
    print(str(self.unique_id) + " : esta actualmente en -> " + str(self.pos) + " - Direction = " + str(self.direction) + " - Visible = " + str(self.isActive))

    # Updates the state, direction and movemente if there has been a change in the next position
    if(self.next_pos and self.next_pos != self.pos):
      self.state = self.next_state
      self.direction = self.next_direction
      self.isActive = self.next_isActive
      self.model.grid.move_agent(self, self.next_pos)

      # Clears the current street state for the next step (since the car is going to move to the next position)
      if(self.streetOn and (self.state != 2 or self.waiting_park_time > 0)):
        self.streetOn.state = 0

        if self.adjacent_street_to_parking_spot_free and self.firstNeighbor:
          self.firstNeighbor.state = 0

      


# The Admin Agent is responsible of finding the closest parking spot 
# for the cars.
class Admin(Agent):
  # Constructor of the Admin class
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.first_car_in_line = None
  
  # Step function of the Admin class. It calls the closest_parking_spot function
  # to determine the closest available parking spot's coordinates.
  def step(self):
    neighbors = self.model.grid.get_neighbors(
        (self.pos),
        moore = False,
        include_center = False,
    )
    for neighbor in neighbors:
      if isinstance(neighbor, Car) and neighbor.pos == (3,1) and neighbor.parking_coordinates == (0,0):
        self.first_car_in_line = neighbor
        closest_parking = closest_parking_spot(self.first_car_in_line, self.model)
        print(f'{neighbor.unique_id} de color: {neighbor.color} tiene sus closest coord en {closest_parking} ')
        self.first_car_in_line.parking_coordinates = closest_parking
        break
      

# The Plaza class is the Model class for this multiagent system.
# It creates the MultiGrid and utilizes a scheduler with SimultaneousActivation.
# It also creates all the agents used in the system, including cars, an admin, 
# parking spots, streets and blocked cells.
class Plaza(Model):
  """ Model of a plaza with a parking lot. """

  # Constructor of the Plaza model
  def __init__(self, cars):
    self.num_cars = cars
    self.grid = MultiGrid(7, 16, False) # False: no torus wrap
    self.schedule = SimultaneousActivation(self)

    # Number of iterations
    self.num = 0;

    # Create Street Agents
    for i in range(len(streets)):
      s = Street("Street" + str(i), self)
      self.schedule.add(s)
      self.grid.place_agent(s, streets[i]) 

    # Create Car agents
    for i in range(self.num_cars):
      c = Car('Car' + str(i), self)
      self.schedule.add(c)
      self.grid.place_agent(c, (3,0)) 

    # Create Admin agent
    a = Admin('Admin1', self)
    self.schedule.add(a)
    self.grid.place_agent(a, (2,1))
  
    # Create BlockedCells agent (just for visualization purposes)
    for i in range(len(blocked_cells)):
      b = BlockedCell("Blocked" + str(i), self)
      self.schedule.add(b)
      self.grid.place_agent(b, blocked_cells[i]) 

    # Counters of parking spots per section
    cont_A = 0 
    cont_B = 0 
    cont_C = 0  
    cont_D = 0  
    p = None

    # Create parking spots
    for (content, x, y) in self.grid.coord_iter():
      if x == 1 or x == 2:
        if y >= 4 and y <= 7:
          cont_A += 1
          p = ParkingSpot('A' + str(cont_A), self, 'A')
          self.schedule.add(p)
          self.grid.place_agent(p, (x, y))
        elif y >= 8 and y <= 11:
          cont_B += 1
          p = ParkingSpot('B' + str(cont_B), self, 'B')
          self.schedule.add(p)
          self.grid.place_agent(p, (x, y))
      elif x == 4 or x == 5:
        if y >= 4 and y <= 7:
          cont_C += 1
          p = ParkingSpot('C' + str(cont_C), self, 'C')
          self.schedule.add(p)
          self.grid.place_agent(p, (x, y))
        elif y >= 8 and y <= 11:
          cont_D += 1
          p = ParkingSpot('D' + str(cont_D), self, 'D')
          self.schedule.add(p)
          self.grid.place_agent(p, (x, y)) 
    
    
    # Data collector
    self.datacollector = DataCollector(
        model_reporters={"Grid": get_status_grid}
    )

  # Step of the Plaza class, it collects the model's data to use it to create
  #  a visualization and schedules the entire model's step 
  def step(self):
    print(f"Iteracion {self.num}")
    self.num += 1
    self.datacollector.collect(self)
    self.schedule.step()



In [ ]:
new_model = Plaza(60)
Iterations = 300

for i in range(Iterations):
  new_model.step()

data = new_model.datacollector.get_model_vars_dataframe()

Se han truncado las últimas 5000 líneas del flujo de salida.
Car1 : esta actualmente en -> (3, 15) - Direction = 0 - Visible = False
Car2 : esta actualmente en -> (3, 15) - Direction = 0 - Visible = False
Car3 : esta actualmente en -> (3, 15) - Direction = 0 - Visible = False
Car4 : esta actualmente en -> (3, 15) - Direction = 0 - Visible = False
Car5 : esta actualmente en -> (3, 15) - Direction = 0 - Visible = False
Car6 : esta actualmente en -> (3, 15) - Direction = 0 - Visible = False
Car7 : esta actualmente en -> (3, 15) - Direction = 0 - Visible = False
Car8 : esta actualmente en -> (3, 15) - Direction = 0 - Visible = False
Car9 : esta actualmente en -> (3, 15) - Direction = 0 - Visible = False
Car10 : esta actualmente en -> (3, 15) - Direction = 0 - Visible = False
Car11 : esta actualmente en -> (3, 15) - Direction = 0 - Visible = False
Car12 : esta actualmente en -> (3, 15) - Direction = 0 - Visible = False
Car13 : esta actualmente en -> (3, 15) - Direction = 0 - Visible = False

In [ ]:
# Guardar lo que regresa el DataCollector
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
%%capture 
fig, axs = plt.subplots(figsize=(7,7))
patch = plt.imshow(data.iloc[0][0], cmap = plt.cm.binary)

def animate(index):
  patch.set_data(data.iloc[index][0])

anim = animation.FuncAnimation(fig, animate, frames = Iterations)
from IPython.display import HTML

In [ ]:
HTML(anim.to_jshtml())